In [1]:
import tensorflow as tf
import numpy as np
import os
from datetime import datetime

This code combines all the concepts introduced in Chapter 9, namely Modular functions, name scopes, Optimization,
Saving and Restoring models.


In [2]:
n_inputs = 28 * 28 
n_hidden_1 = 300
n_hidden_2 = 100
n_outputs = 10

In [3]:
def neuron_layer(X, n_neuros, name, activation):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neuros), stddev=stddev)
        W = tf.Variable(init, name = 'weights')
        b = tf.Variable(tf.zeros([n_neuros]), name = 'biases')
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z
        

In [4]:
# Construction phase
X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(dtype=tf.int64, shape=(None), name='y')

with tf.name_scope('dnn'):
    hidden_1 = neuron_layer(X, n_hidden_1, "hidden_1", activation='relu')
    hidden_2 = neuron_layer(hidden_1, n_hidden_2, "hidden_2", activation='relu')
    logits = neuron_layer(hidden_2, n_outputs, "outputs", activation = None)

with tf.name_scope('loss'):
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(x_entropy, name='loss')
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


In [5]:
now = datetime.utcnow().strftime("%Y-%m-%d-%H:%M:%S")
root_logdir = "tf_logs"
logdir = os.path.join(os.getcwd(),"{}/run-{}".format(root_logdir, now))

In [6]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [7]:
file_writer.close()